### Parse Sensor Data

In [3]:
import pandas as pd

In [4]:
!cat /Users/timlee/data/sf_parking/prkg_sensor_sample_10.csv

### Import CSV - sample to prototype

In [5]:
snsr = pd.read_csv('/Users/timlee/data/sf_parking/prkg_sensor_sample.csv')

In [6]:
snsr.head()

,BLOCK_ID,STREET_NAME,BLOCK_NUM,STREET_BLOCK,AREA_TYPE,PM_DISTRICT_NAME,RATE,START_TIME_DT,TOTAL_TIME,TOTAL_OCCUPIED_TIME,...,GMP_UNKNOWN_TIME,COMM_TIME,COMM_OCCUPIED_TIME,COMM_VACANT_TIME,COMM_UNKNOWN_TIME,CAL_MONTH_NAME,CAL_YEAR,CAL_DATE,DAY_TYPE,TIME_OF_DAY
0,41522,FILLMORE ST,22,FILLMORE ST 2200,Pilot,Fillmore,NaN,01-jul-2012 21:00:00,61200,35847,...,0,0,0,0,0,July,2012,01-jul-2012,weekend,2100
1,41524,FILLMORE ST,24,FILLMORE ST 2400,Pilot,Fillmore,NaN,01-jul-2012 22:00:00,64800,27576,...,0,0,0,0,0,July,2012,01-jul-2012,weekend,2200
2,50001,JACKSON ST,1,JACKSON ST 100,Pilot,Downtown,NaN,01-jul-2012 02:00:00,79200,39985,...,0,0,0,0,0,July,2012,01-jul-2012,weekend,200
3,50002,JACKSON ST,2,JACKSON ST 200,Pilot,Downtown,NaN,01-jul-2012 08:00:00,82800,30465,...,0,0,0,0,0,July,2012,01-jul-2012,weekend,800
4,10903,09TH AVE,3,09TH AVE 300,Control,Inner Richmond,NaN,01-jul-2012 08:00:00,93600,85690,...,0,0,0,0,0,July,2012,01-jul-2012,weekend,800


### Swap the BLOCKS and STREET order - test on sample

In [8]:
def reorder_street_block(strr):
    num = strr.split(' ')[-1]
    rest = ' '.join(strr.split(' ')[:-1])
    if rest[0] == '0':
        rest = rest[1:]
    return('%s %s' % (num, rest))

snsr['search'] = snsr['STREET_BLOCK'].map(lambda x : reorder_street_block(x))
snsr.groupby(['search'])['BLOCK_ID'].count().reset_index().head(10)

,search,BLOCK_ID
0,0 1ST ST,273
1,0 2ND ST,239
2,0 AVILA ST,245
3,0 BARTLETT ST,233
4,0 BEALE ST,237
5,0 CLEMENT ST,245
6,0 DAVIS ST,273
7,0 DRUMM ST,296
8,0 EMBARCADERO NORTH,277
9,0 EMBARCADERO SOUTH,274


### Load all the sensor data

In [9]:
all_ = pd.read_csv('/Users/timlee/data/sf_parking/ParkingSensorData.csv', low_memory=False)

In [10]:
all_.shape

(7902291, 33)

### Group by STREET_BLOCK to make the conversion as fast as possible

In [20]:
#all_['search'] = all_['STREET_BLOCK'].map(lambda x : reorder_street_block(x))
street_blocks = all_.groupby(['STREET_BLOCK'])['BLOCK_ID'].count().reset_index()

### Map our reorder street block

In [23]:
street_blocks['search'] = street_blocks['STREET_BLOCK'].map(lambda x : reorder_street_block(x))

### Now have to plug these reordered address into Scraper to get GPS coordinates